# Backpropagation

[Building makemore pt 5: Building a WaveNet](https://www.youtube.com/watch?v=t3YJ5hKiMQ0)

| Date | User | Change Type | Remarks |  
| ---- | ---- | ----------- | ------- |
| 22/10/2025   | Martin | Created   | Notebook to learn about backpropagation | 
| 24/10/2025   | Martin | Update   | Started with individual element derivatives | 
| 27/10/2025   | Martin | Update   | Continued with backpropagation | 
| 28/10/2025   | Martin | Update   | Completed backpropagation | 

# Content

* [Dataset Creation](#dataset-creation)
* [Manual Backpropagation](#manual-backpropogation)

In [60]:
%load_ext watermark

The watermark extension is already loaded. To reload it, use:
  %reload_ext watermark


In [61]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

# Dataset Creation

Same functions as previous section

In [62]:
# Read in all the words
words = open('data/names.txt', 'r').read().splitlines()
words[:8]

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [63]:
# Build the vocabulary of characters and mappings to/from integers
chars = sorted(list(set(''.join(words))))
stoi = {v: k+1 for k, v in enumerate(chars)}
stoi['.'] = 0
itos = {v: k for k, v in stoi.items()}
vocab_size = len(itos)
print(itos)
print(vocab_size)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}
27


In [64]:
def build_dataset(words):
  block_size = 3
  X, Y = [], []
  for w in words:
    context = [0] * block_size
    for ch in w + '.':
      ix = stoi[ch]
      X.append(context)
      Y.append(ix)
      context = context[1:] + [ix]
    
  X = torch.tensor(X)
  Y = torch.tensor(Y)
  print(X.shape, Y.shape)

  return X, Y

import random
random.seed(42)
random.shuffle(words)
n1 = int(0.8 * len(words))
n2 = int(0.9 * len(words))

X_train, y_train = build_dataset(words[:n1])
X_val, y_val = build_dataset(words[n1:n2])
X_test, y_test = build_dataset(words[n2:])

block_size = 3

torch.Size([182625, 3]) torch.Size([182625])
torch.Size([22655, 3]) torch.Size([22655])
torch.Size([22866, 3]) torch.Size([22866])


# Manual Backpropogation

Backpropogating through all of the variables as they are defined in the forward pass

In [65]:
# Function to compare the gradients between manually calculated and torch calculated
def cmp(s, dt, t):
  ex = torch.all(dt == t.grad).item() # Checks if they are exactly the same
  app = torch.allclose(dt, t.grad) # Checks if they are within a tolerance
  maxdiff = (dt - t.grad).abs().max().item()
  print(f"{s:15s} | exact: {str(ex):5s} | approximate: {str(app):5s} | maxdiff: {maxdiff}")

Initialisation of the weights and bias is slightly different by multiplying some small value to each of them. This is because soemtimes initialising with e.g all zeros could mask incorrect implementation of the backward pass

In [66]:
# Initialise weights and biases
n_embd = 10
n_hidden = 64

g = torch.Generator().manual_seed(2147483647)
C = torch.randn((vocab_size, n_embd), generator=g) 
# Layer 1
W1 = torch.randn((n_embd * block_size, n_hidden), generator=g) * (5/3)/((n_embd * block_size) ** 0.5)
b1 = torch.randn(n_hidden, generator=g) * 0.1
# Layer 2
W2 = torch.randn((n_hidden, vocab_size), generator=g) * 0.1
b2 = torch.randn(vocab_size, generator=g) * 0.1

# BatchNorm parameters
bngain = torch.ones((1, n_hidden)) * 0.1 + 1.0
bnbias = torch.ones((1, n_hidden)) * 0.1

parameters = [C, W1, b1, W2, b2, bngain, bnbias]
for p in parameters:
  p.requires_grad = True

print(f"Total number of parameters: {sum(p.nelement() for p in parameters)}")

Total number of parameters: 4137


In [67]:
batch_size = 32
n = batch_size

# Minibatch
ix = torch.randint(0, X_train.shape[0], (batch_size, ), generator=g)
X_batch, y_batch = X_train[ix], y_train[ix]

In [68]:
# Explicit forward pass
emb = C[X_batch]
embcat = emb.view(emb.shape[0], -1)

# Linear layer 1
h_pre_bn = embcat @ W1 + b1 # Hidden layer pre-activation
# BatchNorm layer
bn_mean_i = 1/n * h_pre_bn.sum(0, keepdim=True)
bn_diff = h_pre_bn - bn_mean_i
bn_diff_sq = bn_diff ** 2
bn_var = 1/(n-1) * (bn_diff_sq).sum(0, keepdim=True)
bn_var_inv = (bn_var + 1e-5)**-0.5
bn_raw = bn_diff * bn_var_inv
h_preact = bngain * bn_raw + bnbias
# Non-linearity
h = torch.tanh(h_preact)

# Linear layer 2
logits = h @ W2 + b2
# Cross entropy loss
logit_maxes = logits.max(1, keepdim=True).values
norm_logits = logits - logit_maxes # subtract max for numerical stability
counts = norm_logits.exp()
counts_sum = counts.sum(1, keepdim=True)
counts_sum_inv = counts_sum**-1
probs = counts * counts_sum_inv
logprobs = probs.log()
loss = -logprobs[range(n), y_batch].mean()

# Pytorch backward pass
for p in parameters:
  p.grad = None
for t in [
  logprobs, probs, counts, counts_sum, counts_sum_inv,
  norm_logits, logit_maxes, logits, h, h_preact, bn_raw,
  bn_var, bn_var_inv, bn_diff_sq, bn_diff, h_pre_bn, bn_mean_i,
  embcat, emb
]:
  t.retain_grad()
loss.backward()
loss

tensor(3.3603, grad_fn=<NegBackward0>)

Backpropogation here

- Sizes of the derivatives are always the same as their original tensors: Use the size of the tensors to figure out what to do
- From Math: Derivatives (gradients) will always sum their components

<u>Notes of Interpretation</u>

- `probs`: If the probability of the correct class is low, it's boosting the derivative of the log probs to adjust the weights for the correct class
- `norm_logits`: Gradient of `logit_maxes` should be zero (or close to due to floating point precision). This is because it only scales the values to prevent overflow during the exponent in the subsequent step. Since the output is a softmax, the probabilities don't change, therefore, it should not have any impact on the update step i.e 0 gradient

Additional Matrix Derivatives

![image](./assets/matrix_derivative.png)
![image](./assets/matrix_derivative_2.png)

In [69]:
d_logprobs = torch.zeros_like(logprobs)
d_logprobs[range(n), y_batch] = -1.0/n # derivative
cmp('logprobs', d_logprobs, logprobs)

d_probs = (1.0 / probs) * d_logprobs # chain rule 
cmp('probs', d_probs, probs)

d_counts_sum_inv = (counts * d_probs).sum(1, keepdim=True)
cmp('count_sum_inv', d_counts_sum_inv, counts_sum_inv)

d_counts_sum = (-1.0 / counts_sum**2.0) * d_counts_sum_inv
cmp('counts_sum', d_counts_sum, counts_sum)

# d_counts is being used twice
# 1. probs = counts * counts_sum_inv
# 2. counts_sum = counts.sum(...)
d_counts = d_probs * counts_sum_inv + torch.ones_like(counts) * d_counts_sum
cmp('counts', d_counts, counts)

d_norm_logits = norm_logits.exp() * d_counts
cmp('norm_logits', d_norm_logits, norm_logits)

d_logit_maxes = (-d_norm_logits).sum(1, keepdim=True)
cmp('logit_maxes', d_logit_maxes, logit_maxes)

# 1. norm_logits
# 2. logit_maxes
temp = F.one_hot(logits.max(1).indices, num_classes=logits.shape[1]) * d_logit_maxes # logits_max
d_logits = (d_norm_logits.clone()) + temp
cmp('logits', d_logits, logits)


# Linear layer 2
d_h = d_logits @ W2.T
d_W2 = h.T @ d_logits
d_b2 = d_logits = d_logits.sum(0)
cmp('h', d_h, h)
cmp('W2', d_W2, W2)
cmp('b2', d_b2, b2)


# Non-linearity
d_h_preact = (1.0 - h**2)  * d_h
cmp('h_preact', d_h_preact, h_preact)


# Batch Normalisation
d_bngain = (bn_raw * d_h_preact).sum(0, keepdim=True)
d_bn_raw = bngain * d_h_preact
d_bnbias = d_h_preact.sum(0, keepdim=True)
cmp('bngain', d_bngain, bngain)
cmp('bn_raw', d_bn_raw, bn_raw)
cmp('bnbias', d_bnbias, bnbias)

d_bn_var_inv = (bn_diff * d_bn_raw).sum(0, keepdim=True)
cmp('bn_var_inv', d_bn_var_inv, bn_var_inv)

# d_bn_var = (-0.5) * (1.0/(bn_var + 1e-5)**(3/2)) * d_bn_var_inv
d_bn_var = (-0.5*(bn_var + 1e-5)**-1.5) * d_bn_var_inv
cmp('bn_var', d_bn_var, bn_var)

d_bn_diff_sq = (1.0/(n-1)) * torch.ones_like(bn_diff_sq) * d_bn_var
cmp('bn_diff_sq', d_bn_diff_sq, bn_diff_sq)

# 1. bn_raw = bn_diff * bn_var_inv
# 2. dn_diff_sq = bn_diff**2
temp = 2 * bn_diff * d_bn_diff_sq
d_bn_diff = bn_var_inv * d_bn_raw + temp
cmp('bn_diff', d_bn_diff, bn_diff)

d_bn_mean_i = (-d_bn_diff).sum(0)
cmp('bn_mean_i', d_bn_mean_i, bn_mean_i)

# 1. bn_diff = h_pre_bn - bn_mean_i
# 2. bn_mean_i = 1/n * h_pre_bn.sum(0, keepdim=True)
d_h_pre_bn = d_bn_diff.clone() + torch.ones_like(h_pre_bn) * 1.0/n * d_bn_mean_i
cmp('h_pre_bn', d_h_pre_bn, h_pre_bn)


# Linear Layer 1
d_embcat = d_h_pre_bn @ W1.T
d_W1 = embcat.T @ d_h_pre_bn
d_b1 = d_h_pre_bn.sum(0)
cmp('embcat', d_embcat, embcat)
cmp('W1', d_W1, W1)
cmp('b1', d_b1, b1)


# Embeddings - re-represent the shape
d_emb = d_embcat.view(emb.shape)
cmp('emb', d_emb, emb)

d_C = torch.zeros_like(C)
for i in range(X_batch.shape[0]):
  for j in range(X_batch.shape[1]):
    idx = X_batch[i, j]
    d_C[idx] += d_emb[i, j]
cmp('C', d_C, C)

logprobs        | exact: True  | approximate: True  | maxdiff: 0.0
probs           | exact: True  | approximate: True  | maxdiff: 0.0
count_sum_inv   | exact: True  | approximate: True  | maxdiff: 0.0
counts_sum      | exact: True  | approximate: True  | maxdiff: 0.0
counts          | exact: True  | approximate: True  | maxdiff: 0.0
norm_logits     | exact: True  | approximate: True  | maxdiff: 0.0
logit_maxes     | exact: True  | approximate: True  | maxdiff: 0.0
logits          | exact: True  | approximate: True  | maxdiff: 0.0
h               | exact: True  | approximate: True  | maxdiff: 0.0
W2              | exact: True  | approximate: True  | maxdiff: 0.0
b2              | exact: True  | approximate: True  | maxdiff: 0.0
h_preact        | exact: True  | approximate: True  | maxdiff: 0.0
bngain          | exact: True  | approximate: True  | maxdiff: 0.0
bn_raw          | exact: True  | approximate: True  | maxdiff: 0.0
bnbias          | exact: True  | approximate: True  | maxdiff:

---

# Backpropagation on Combined Expressions

Instead of breaking formulas up into individual pieces, the combined computation when differentiated can be simplified

Example: Using Cross Entropy to compute the loss

In [70]:
loss_fast = F.cross_entropy(logits, y_batch)
print(loss_fast.item(), 'diff:', (loss_fast - loss).item())

3.36032772064209 diff: 2.384185791015625e-07


Derivation of the the derivative of cross entropy

![image](./assets/cross_entropy_derivative.png)

In [71]:
# Backward pass for Softmax
d_logits = F.softmax(logits, 1)
d_logits[range(n), y_batch] -= 1
d_logits /= n

cmp('logits', d_logits, logits)

logits          | exact: False | approximate: True  | maxdiff: 6.05359673500061e-09


Derivation of derivative for Batch Normalisation

![image](./assets/batch_normalisation_derivative.png)

In [72]:
# Not easy to get this formula, because the formula above only does it for one neuron
# But there are 64 neurons that must all be done in parallel to ensure the shapes match
d_h_pre_bn = bngain * bn_var_inv/n * (n*d_h_preact - d_h_preact.sum(0) - n/(n-1)*bn_raw*(d_h_preact*bn_raw).sum(0))
cmp('h_pre_bn', d_h_pre_bn, h_pre_bn)

h_pre_bn        | exact: False | approximate: True  | maxdiff: 9.313225746154785e-10


---

# Putting it all together

In [90]:
# Initialise weights and biases
n_embd = 10
n_hidden = 64

g = torch.Generator().manual_seed(2147483647)
C = torch.randn((vocab_size, n_embd), generator=g) 
# Layer 1
W1 = torch.randn((n_embd * block_size, n_hidden), generator=g) * (5/3)/((n_embd * block_size) ** 0.5)
b1 = torch.randn(n_hidden, generator=g) * 0.1
# Layer 2
W2 = torch.randn((n_hidden, vocab_size), generator=g) * 0.1
b2 = torch.randn(vocab_size, generator=g) * 0.1

# BatchNorm parameters
bngain = torch.ones((1, n_hidden)) * 0.1 + 1.0
bnbias = torch.ones((1, n_hidden)) * 0.1

parameters = [C, W1, b1, W2, b2, bngain, bnbias]
for p in parameters:
  p.requires_grad = True

print(f"Total number of parameters: {sum(p.nelement() for p in parameters)}")

# Model
max_steps = 200_000
batch_size = 32
lossi = []

for i in range(max_steps):
  # minibatch
  ix = torch.randint(0, X_train.shape[0], (batch_size,), generator=g)
  X_batch, y_batch = X_train[ix], y_train[ix]

  # forward pass
  emb = C[X_batch]
  embcat = emb.view(emb.shape[0], -1)
  hpreact = embcat @ W1 # Bias was removed
  bnmeani = hpreact.mean(0, keepdim=True)
  bnstdi = hpreact.std(0, keepdim=True)

  hpreact = bngain * (hpreact - bnmeani) / bnstdi + bnbias
  h = torch.tanh(hpreact)
  logits = h @ W2 + b2
  loss = F.cross_entropy(logits, y_batch)

  # backward pass
  for p in parameters:
    p.grad = None
  loss.backward()

  # ---------- Manual Backprop ----------
  dlogits = F.softmax(logits, 1)
  dlogits[range(n), y_batch] -= 1
  dlogits /= n
  # 2nd layer backprop
  dh = dlogits @ W2.T
  dW2 = h.T @ dlogits
  db2 = dlogits.sum(0)
  # tanh
  dhpreact = (1.0 - h**2) * dh
  # batchnorm backprop
  dbngain = (bn_raw * dhpreact).sum(0, keepdim=True)
  dbnbias = dhpreact.sum(0, keepdim=True)
  dhprebn = bngain*bn_var_inv/n * (n*dhpreact - dhpreact.sum(0) - n/(n-1)*bn_raw*(dhpreact*bn_raw).sum(0))
  # 1st layer
  dembcat = dhprebn @ W1.T
  dW1 = embcat.T @ dhprebn
  db1 = dhprebn.sum(0)
  # embedding
  demb = dembcat.view(emb.shape)
  dC = torch.zeros_like(C)
  for k in range(X_batch.shape[0]):
    for j in range(X_batch.shape[1]):
      ix = X_batch[k,j]
      dC[ix] += demb[k,j]
  grads = [dC, dW1, db1, dW2, db2, dbngain, dbnbias] 
  # -------------------------------------

  # Update
  lr = 0.1 if i < 100_000 else 0.01
  for p, grad in zip(parameters, grads):
    p.data += -lr * grad
    # p.data += -lr * p.grad
  
  # Track stats
  if i % 10_000 == 0:
    print(f"{i:7d}/{max_steps:7d}: {loss.item():.4f}")
  lossi.append(loss.log10().item())

  # if i >= 100:
  #   break

Total number of parameters: 4137
      0/ 200000: 3.3603
  10000/ 200000: 2.4803
  20000/ 200000: 2.1109
  30000/ 200000: 2.3620
  40000/ 200000: 2.3571
  50000/ 200000: 2.2323
  60000/ 200000: 2.2413
  70000/ 200000: 2.4117
  80000/ 200000: 2.1686
  90000/ 200000: 2.4417
 100000/ 200000: 2.1538
 110000/ 200000: 1.9201
 120000/ 200000: 2.0154
 130000/ 200000: 2.6449
 140000/ 200000: 1.9107
 150000/ 200000: 2.2465
 160000/ 200000: 2.1470
 170000/ 200000: 2.2498
 180000/ 200000: 2.0757
 190000/ 200000: 2.0521


In [78]:
with torch.no_grad():
  emb = C[X_train]
  embcat = emb.view(emb.shape[0], -1)
  hpreact = embcat @ W1 + b1
  bnmean = hpreact.mean(0, keepdim=True)
  bnvar = hpreact.var(0, keepdim=True, unbiased=True)

In [84]:
@torch.no_grad()
def split_loss(split):
  x, y = {
    'train': (X_train, y_train),
    'val': (X_val, y_val),
    'test': (X_test, y_test)
  }[split]

  emb = C[x]
  embcat = emb.view(emb.shape[0], -1)
  hpreact = embcat @ W1 + b1
  # hpreact = bngain * (hpreact - hpreact.mean(0, keepdim=True)) / hpreact.std(0, keepdim=True) + bnbias
  hpreact = bngain * (hpreact - bnmean) * (bnvar + 1e-5)**-0.5 + bnbias
  h = torch.tanh(hpreact)
  logits = h @ W2 + b2
  loss = F.cross_entropy(logits, y)
  print(split, loss.item())

split_loss('train')
split_loss('val')

train 7.0931854248046875
val 7.030697345733643


In [ ]:
%watermark